Import all necessary and relevant libraries 

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Get the data from the internet and use beautiful soup to parse the codes

In [2]:
wikipedia_link='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.'
raw_wikipedia_page= requests.get(wikipedia_link).text
soup = BeautifulSoup(raw_wikipedia_page,'xml')

Extract the raw table in the webpage and clean the data

In [3]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

Integrate postal codes with more than 1 neighbor

In [4]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 78


create the pandas dataframe

In [5]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.to_csv('toronto_part1.csv')
df_toronto.head(14)

,Postcode,Borough,Neighbourhood
0,M2L,North York,York Mills
1,M2M,North York,"Newtonbrook, Willowdale"
2,M4Y,Downtown Toronto,Church and Wellesley
3,M3A,North York,Parkwoods
4,M8W,Etobicoke,"Alderwood, Long Branch"
5,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre"
6,M5C,Downtown Toronto,St. James Town
7,M1N,Scarborough,Birch Cliff
8,M3H,North York,"Bathurst Manor, Wilson Heights"
9,M8V,Etobicoke,New Toronto


In [6]:
df_toronto.shape

(78, 3)